In [ ]:
!pip install -q \
  llama-index \
  llama-index-embeddings-huggingface \
  llama-index-llms-huggingface \
  transformers accelerate sentence-transformers pypdf bitsandbytes


In [2]:
!pip install -q gradio

In [ ]:
from google.colab import files

uploaded_files = files.upload()


In [ ]:
from pypdf import PdfReader
from llama_index.core import Document

documents = []

for filename in uploaded_files:
    reader = PdfReader(filename)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""

    if text.strip():
        documents.append(Document(text=text))

print(f"Loaded {len(documents)} document(s)")


In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-base-en-v1.5"
)


In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

Settings.llm = HuggingFaceLLM(
    model_name="Qwen/Qwen2-7B-Instruct",
    tokenizer_name="Qwen/Qwen2-7B-Instruct",
    device_map="auto",
    model_kwargs={
        "load_in_4bit": True,
        "bnb_4bit_compute_dtype": torch.float16,
        "bnb_4bit_use_double_quant": True,
        "bnb_4bit_quant_type": "nf4",
    },
    generate_kwargs={
        "temperature": 0.1
    },
    is_chat_model=True,
)


In [7]:
Settings.chunk_size = 512
Settings.chunk_overlap = 50


In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

print("Vector index built successfully")


In [9]:
from llama_index.core.prompts import PromptTemplate

LEGAL_PROMPT = PromptTemplate(
"""
Answer ONLY using the provided context.
If the answer is not present, reply exactly:
"The uploaded document does not contain information relevant to this query."

Context:
{context_str}

Question:
{query_str}

Answer:
"""
)


In [10]:
from llama_index.core.response_synthesizers import ResponseMode

query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode=ResponseMode.REFINE,
    text_qa_template=LEGAL_PROMPT,
)


In [11]:
def chat_with_lexibot(user_query):
    if not user_query.strip():
        return "Please enter a valid question."

    answer = str(query_engine.query(user_query)).strip()

    if len(answer) < 40:
        return "The uploaded document does not contain information relevant to this query."

    return answer



In [ ]:
import gradio as gr

with gr.Blocks(title="LexiBot ⚖️") as demo:
    gr.Markdown(
        """
        # ⚖️ LexiBot
        **Legal Document Assistant**
        Upload documents above, then ask legal questions below.
        """
    )

    chatbot = gr.Chatbot(height=350)

    user_input = gr.Textbox(
        placeholder="Ask a legal question...",
        show_label=False
    )

    def respond(message, history):
        response = chat_with_lexibot(message)
        history.append((message, response))
        return history, ""

    user_input.submit(respond, [user_input, chatbot], [chatbot, user_input])

demo.launch()
